<a href="https://colab.research.google.com/github/saakolch/question-answering_model/blob/main/Q-A_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate transformers sentencepiece

Let's do tokenize

In [32]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

checkpoints = 'distilbert-base-cased-distilled-squad'
tokenizer = AutoTokenizer.from_pretrained(checkpoints)
model = AutoModelForQuestionAnswering.from_pretrained(checkpoints)


content = '''

Грани правильного октаэдра раскрашены в белый и черный цвет. При этом любые две грани, имеющие общее ребро, покрашены в разные цвета.
Докажите, что для любой точки внутри октаэдра сумма расстояний до плоскостей белых граней равна сумме расстояний до плоскостей черных граней

'''

question = 'Какая тема этой задачи?'

inputs = tokenizer(question, content, return_tensors='pt')
outputs = model(**inputs)

In [33]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits
start_logits.shape, end_logits.shape

(torch.Size([1, 254]), torch.Size([1, 254]))

We have to equate a [CLS] and [SEP] to number (a large negative number) for preparing to process of softmax using

In [34]:
import torch

sequence_ids = inputs.sequence_ids()
mask = [i != 1 for i in sequence_ids]
mask[0] = False
mask = torch.tensor(mask)[None]

start_logits[mask] = -10000
end_logits[mask] = -10000

In [35]:
start_prob = torch.nn.functional.softmax(start_logits, dim=-1)[0]
end_prob = torch.nn.functional.softmax(end_logits, dim=-1)[0]

As we know the NLP tasks and especially question-answering task is a code + probability of appearing of words, based on previous information. So we are going to do a simple math here. We want to predict the start of the entity and the end of one. Assuming we have independent events such us answer in the text with probability at the start and probability at the end, therefore we need to multiple each prob of start-end and take the argmax of all of them

In [36]:
scores = start_prob[:, None] * end_prob[None,:]
scores.shape

torch.Size([254, 254])

In [37]:
scores = torch.triu(scores)
scores.shape

torch.Size([254, 254])

In [38]:
max_index = scores.argmax().item()
start_index = max_index // scores.shape[1]
end_index = max_index % scores.shape[1]
scores[start_index, end_index]

tensor(0.3980, grad_fn=<SelectBackward0>)